In [1]:
import pandas as pd
import numpy as np

In [4]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)

In [5]:
basics_df =  basics_df.drop('endYear',axis=1)
movies = basics_df.loc[basics_df['titleType']=='movie']
movies = movies.drop('titleType',axis=1)
movies_ok = movies.loc[movies['isAdult']=='0']
movies_ok=movies_ok.drop('isAdult',axis=1)
movies_ok_nat = movies_ok.replace('\\N', np.nan)
movies_clean = movies_ok_nat.dropna()
movies_clean = movies_clean.astype({'runtimeMinutes': int})
movies_clean = movies_clean.astype({'startYear': int})
movies_clean = movies_clean[(movies_clean['runtimeMinutes'] >= 58) & (movies_clean['runtimeMinutes'] <= 270)]
movies_clean = movies_clean[(movies_clean['startYear'] >= 1918) & (movies_clean['startYear'] <= 2021)]

In [6]:
movies_clean.head()

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
2396,tt0002423,Passion,Madame DuBarry,1919,85,"Biography,Drama,Romance"
5013,tt0005076,Charley's Aunt,Charley's Aunt,1925,80,Comedy
6979,tt0007068,Mind the Paint Girl,Mind the Paint Girl,1919,60,"Drama,Romance"
7328,tt0007422,Das Tagebuch des Dr. Hart,Das Tagebuch des Dr. Hart,1918,82,"Drama,Romance"
7449,tt0007547,Der Weg des Todes,Der Weg des Todes,1918,75,Drama


In [7]:
ratings_df = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t")

In [8]:
movies = pd.merge(movies_clean,ratings_df,how='inner')
movies_7 = movies.loc[movies['averageRating']>=7.0]
movies_7_rating = movies_7[movies_7['numVotes'] >= 1000]

In [9]:
movies_7_rating.head()

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
11,tt0008879,The Outlaw and His Wife,Berg-Ejvind och hans hustru,1918,136,Drama,7.2,1784
82,tt0009893,The Oyster Princess,Die Austernprinzessin,1919,60,Comedy,7.2,1817
88,tt0009937,Blind Husbands,Blind Husbands,1919,99,"Drama,Romance",7.0,1470
91,tt0009968,Broken Blossoms,Broken Blossoms or The Yellow Man and the Girl,1919,90,"Drama,Romance",7.3,10029
124,tt0010247,Sir Arne's Treasure,Herr Arnes pengar,1919,122,"Drama,History",7.2,1357


In [10]:
movie_for_ml = movies_7_rating.drop(columns=['primaryTitle','runtimeMinutes','averageRating','numVotes'],axis =1)

In [12]:
movie_for_ml.reset_index(drop=True, inplace=True)
movie_for_ml

,tconst,originalTitle,startYear,genres
0,tt0008879,Berg-Ejvind och hans hustru,1918,Drama
1,tt0009893,Die Austernprinzessin,1919,Comedy
2,tt0009937,Blind Husbands,1919,"Drama,Romance"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,"Drama,Romance"
4,tt0010247,Herr Arnes pengar,1919,"Drama,History"
...,...,...,...,...
11649,tt9882084,Chasing Happiness,2019,"Biography,Documentary,Music"
11650,tt9886872,Munthiri Monchan,2019,"Comedy,Romance"
11651,tt9900782,Kaithi,2019,"Action,Crime,Thriller"
11652,tt9902160,Herself,2020,Drama


In [15]:
movie_for_ml = movie_for_ml.astype({'startYear': str})
movie_for_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11654 entries, 0 to 11653
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tconst         11654 non-null  object
 1   originalTitle  11654 non-null  object
 2   startYear      11654 non-null  object
 3   genres         11654 non-null  object
dtypes: object(4)
memory usage: 364.3+ KB


In [16]:
movie_for_ml['data'] = movie_for_ml['startYear'] + ',' + movie_for_ml['genres']
movie_for_ml

,tconst,originalTitle,startYear,genres,data
0,tt0008879,Berg-Ejvind och hans hustru,1918,Drama,"1918,Drama"
1,tt0009893,Die Austernprinzessin,1919,Comedy,"1919,Comedy"
2,tt0009937,Blind Husbands,1919,"Drama,Romance","1919,Drama,Romance"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,"Drama,Romance","1919,Drama,Romance"
4,tt0010247,Herr Arnes pengar,1919,"Drama,History","1919,Drama,History"
...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,2019,"Biography,Documentary,Music","2019,Biography,Documentary,Music"
11650,tt9886872,Munthiri Monchan,2019,"Comedy,Romance","2019,Comedy,Romance"
11651,tt9900782,Kaithi,2019,"Action,Crime,Thriller","2019,Action,Crime,Thriller"
11652,tt9902160,Herself,2020,Drama,"2020,Drama"


In [17]:
movie_for_ml.drop(columns=['startYear','genres'],axis=1,inplace=True)
movie_for_ml

,tconst,originalTitle,data
0,tt0008879,Berg-Ejvind och hans hustru,"1918,Drama"
1,tt0009893,Die Austernprinzessin,"1919,Comedy"
2,tt0009937,Blind Husbands,"1919,Drama,Romance"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,Drama,Romance"
4,tt0010247,Herr Arnes pengar,"1919,Drama,History"
...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,Biography,Documentary,Music"
11650,tt9886872,Munthiri Monchan,"2019,Comedy,Romance"
11651,tt9900782,Kaithi,"2019,Action,Crime,Thriller"
11652,tt9902160,Herself,"2020,Drama"


In [19]:
temp_clusters = pd.read_csv('../data/movies_for_ml.csv.zip')
temp_clusters

,tconst,originalTitle,startYear,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,cluster
0,tt0008879,Berg-Ejvind och hans hustru,-2.974130,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,5
1,tt0009893,Die Austernprinzessin,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,1.587382,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,24
2,tt0009937,Blind Husbands,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,4
4,tt0010247,Herr Arnes pengar,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,1.070327,-0.391523,-0.325588,-0.215457,3.179564,-0.629968,-0.436904,2.801589,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,27
11650,tt9886872,Munthiri Monchan,1.070327,-0.391523,-0.325588,-0.215457,-0.314509,1.587382,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,29
11651,tt9900782,Kaithi,1.070327,2.554127,-0.325588,-0.215457,-0.314509,-0.629968,2.288833,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,3.138634,-0.214802,-0.11648,2
11652,tt9902160,Herself,1.110371,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,5


In [28]:
movie_for_ml['cluster'] = temp_clusters['cluster']
movie_for_ml['data'] = movie_for_ml['data'].apply(str.lower)
movie_for_ml


,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5
1,tt0009893,Die Austernprinzessin,"1919,comedy",24
2,tt0009937,Blind Husbands,"1919,drama,romance",4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18
...,...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,biography,documentary,music",27
11650,tt9886872,Munthiri Monchan,"2019,comedy,romance",29
11651,tt9900782,Kaithi,"2019,action,crime,thriller",2
11652,tt9902160,Herself,"2020,drama",5


In [29]:
movie_for_ml.to_csv('../data/movies_merged.csv.zip',index=False, compression='zip')

In [30]:
movies_merged = pd.read_csv('../data/movies_merged.csv.zip')
movies_merged

,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5
1,tt0009893,Die Austernprinzessin,"1919,comedy",24
2,tt0009937,Blind Husbands,"1919,drama,romance",4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18
...,...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,biography,documentary,music",27
11650,tt9886872,Munthiri Monchan,"2019,comedy,romance",29
11651,tt9900782,Kaithi,"2019,action,crime,thriller",2
11652,tt9902160,Herself,"2020,drama",5
